In [301]:
import pandas as pd
import numpy as np

from sklearn.metrics import confusion_matrix,roc_curve,auc,accuracy_score,classification_report
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [319]:
#reading the data
flights_data1 = pd.read_csv("train1.csv")
flights_data2 = pd.read_csv("train2.csv")
flights_data3 = pd.read_csv("train3.csv")
flights_data4 = pd.read_csv("train4.csv")
flights_data5 = pd.read_csv("train5.csv")

test_data = pd.read_csv("test-1583918175355.csv")


In [320]:
for i in [flights_data2,flights_data3,flights_data4,flights_data5]:
    i.drop(columns='Unnamed: 0',axis=1,inplace=True)

In [321]:
list1 = [flights_data1,flights_data2,flights_data3,flights_data4,flights_data5]
flight_data = pd.concat(list1,ignore_index=True)
flight_data.head()
flight_data.tail()

,FL_DATE,OP_UNIQUE_CARRIER,OP_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN,DEST_AIRPORT_ID,DEST,CRS_DEP_TIME,DEP_TIME,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY_GROUP,CANCELLED,DISTANCE
172189,2019-11-03,AS,AS,N293AK,468,12892,LAX,12478,JFK,1515,1508.0,24.0,1532.0,2320.0,17.0,2345,2337.0,early_arrival,0.0,2475.0
172190,2019-11-19,9E,9E,N305PQ,5298,11433,DTW,11995,GSO,1515,1510.0,14.0,1524.0,1637.0,6.0,1700,1643.0,early_arrival,0.0,460.0
172191,2019-11-25,DL,DL,N590NW,1887,14747,SEA,12173,HNL,1205,1201.0,32.0,1233.0,1547.0,4.0,1629,1551.0,early_arrival,0.0,2677.0
172192,2019-11-25,AX,AX,N41104,4603,13930,ORD,13367,MLI,1930,1923.0,26.0,1949.0,2018.0,2.0,2042,2020.0,early_arrival,0.0,139.0
172193,2019-11-21,MQ,MQ,N264NN,4202,11298,DFW,11986,GRR,1545,1543.0,13.0,1556.0,1852.0,31.0,1908,1923.0,ontime,0.0,931.0


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(9, 8))
sns.heatmap(data = flight_data,cmap='BrBG', annot=True, linewidths=0.2)

KeyboardInterrupt: 

Exception ignored in: 'zmq.backend.cython.message.Frame.__dealloc__'
Traceback (most recent call last):
  File "zmq/backend/cython/checkrc.pxd", line 12, in zmq.backend.cython.checkrc._check_rc
    PyErr_CheckSignals()
KeyboardInterrupt


In [322]:
import datetime
flight_data["FL_DATE"]=pd.to_datetime(flight_data["FL_DATE"])
#flight_data["Year"]=pd.DatetimeIndex(flight_data["FL_DATE"]).year
#flight_data["Month"]=pd.DatetimeIndex(flight_data["FL_DATE"]).month

In [ ]:
flight_data.dtypes

In [313]:
flight_data.dropna(axis=0,inplace=True)

In [ ]:

flight_data.ARR_DELAY_GROUP

In [139]:
flight_data.columns

Index(['FL_DATE', 'OP_UNIQUE_CARRIER', 'OP_CARRIER', 'TAIL_NUM',
       'OP_CARRIER_FL_NUM', 'ORIGIN_AIRPORT_ID', 'ORIGIN', 'DEST_AIRPORT_ID',
       'DEST', 'CRS_DEP_TIME', 'DEP_TIME', 'TAXI_OUT', 'WHEELS_OFF',
       'WHEELS_ON', 'TAXI_IN', 'CRS_ARR_TIME', 'ARR_TIME', 'ARR_DELAY_GROUP',
       'CANCELLED', 'DISTANCE', 'Year', 'Month'],
      dtype='object')

In [ ]:
test-1583918175355.csv

### As the features "OP_UNIQUE_CARRIER" and "OP_CARRIER" are same removing "OP_UNIQUE_CARRIER"

In [323]:
flight_data.drop(columns="OP_UNIQUE_CARRIER",inplace=True,axis=1)

In [84]:
flight_data.columns

Index(['FL_DATE', 'OP_CARRIER', 'TAIL_NUM', 'OP_CARRIER_FL_NUM',
       'ORIGIN_AIRPORT_ID', 'ORIGIN', 'DEST_AIRPORT_ID', 'DEST',
       'CRS_DEP_TIME', 'DEP_TIME', 'TAXI_OUT', 'WHEELS_OFF', 'WHEELS_ON',
       'TAXI_IN', 'CRS_ARR_TIME', 'ARR_TIME', 'ARR_DELAY_GROUP', 'CANCELLED',
       'DISTANCE'],
      dtype='object')

In [196]:
flight_data.ARR_DELAY_GROUP.value_counts()

-1.0    111777
 0.0     38131
 1.0     19410
Name: ARR_DELAY_GROUP, dtype: int64

In [324]:
flight_data.ARR_DELAY_GROUP.replace({"early_arrival":-1,"ontime":0,"delayed":1},inplace=True)   

In [180]:
list(flight_data.columns)

['FL_DATE',
 'OP_UNIQUE_CARRIER',
 'OP_CARRIER',
 'TAIL_NUM',
 'OP_CARRIER_FL_NUM',
 'ORIGIN_AIRPORT_ID',
 'ORIGIN',
 'DEST_AIRPORT_ID',
 'DEST',
 'CRS_DEP_TIME',
 'DEP_TIME',
 'TAXI_OUT',
 'WHEELS_OFF',
 'WHEELS_ON',
 'TAXI_IN',
 'CRS_ARR_TIME',
 'ARR_TIME',
 'ARR_DELAY_GROUP',
 'CANCELLED',
 'DISTANCE',
 'Year',
 'Month']

In [325]:
cols_to_keep = ['FL_DATE','OP_UNIQUE_CARRIER','OP_CARRIER','TAIL_NUM','OP_CARRIER_FL_NUM','ORIGIN_AIRPORT_ID','ORIGIN',"DEST_AIRPORT_ID","DEST",'CRS_DEP_TIME','CRS_ARR_TIME','DISTANCE','ARR_DELAY_GROUP']
cols_to_remove = list(set(flight_data.columns)-set(cols_to_keep))
flight_data.drop(columns=cols_to_remove,axis=1,inplace=True)


In [326]:

flight_data.columns

Index(['FL_DATE', 'OP_CARRIER', 'TAIL_NUM', 'OP_CARRIER_FL_NUM',
       'ORIGIN_AIRPORT_ID', 'ORIGIN', 'DEST_AIRPORT_ID', 'DEST',
       'CRS_DEP_TIME', 'CRS_ARR_TIME', 'ARR_DELAY_GROUP', 'DISTANCE'],
      dtype='object')

In [327]:
flight_data=flight_data[['FL_DATE','OP_CARRIER','TAIL_NUM','OP_CARRIER_FL_NUM','ORIGIN_AIRPORT_ID','ORIGIN',"DEST_AIRPORT_ID","DEST",'CRS_DEP_TIME','CRS_ARR_TIME','DISTANCE','ARR_DELAY_GROUP']]
flight_data.shape[1]

flight_data.dtypes


FL_DATE              datetime64[ns]
OP_CARRIER                   object
TAIL_NUM                     object
OP_CARRIER_FL_NUM             int64
ORIGIN_AIRPORT_ID             int64
ORIGIN                       object
DEST_AIRPORT_ID               int64
DEST                         object
CRS_DEP_TIME                  int64
CRS_ARR_TIME                  int64
DISTANCE                    float64
ARR_DELAY_GROUP             float64
dtype: object

In [202]:
flight_data["FL_DATE"]=pd.to_datetime(flight_data["FL_DATE"])

In [205]:
flight_data.dtypes

FL_DATE              datetime64[ns]
OP_UNIQUE_CARRIER            object
OP_CARRIER                   object
TAIL_NUM                     object
OP_CARRIER_FL_NUM             int64
ORIGIN_AIRPORT_ID             int64
ORIGIN                       object
DEST_AIRPORT_ID               int64
DEST                         object
CRS_DEP_TIME                  int64
CRS_ARR_TIME                  int64
DISTANCE                    float64
ARR_DELAY_GROUP             float64
dtype: object

# Remove the following columns

 ### 1. OP_UNIQUE_CARRIER: 
                 As there is another column named 'OP_CARRIER' which contains the same names.
### 2. ORIGIN_AIRPORT_ID & DEST_AIRPORT_ID : 
                 As the columns 'ORIGIN' & 'DEST' has airport name which has unique id, so any one of the column is enough for our prediction
 

In [330]:
flight_data.drop(columns=["ORIGIN_AIRPORT_ID","DEST_AIRPORT_ID"],axis=1,inplace=True)

In [331]:
flight_data.columns

Index(['FL_DATE', 'OP_CARRIER', 'TAIL_NUM', 'OP_CARRIER_FL_NUM', 'ORIGIN',
       'DEST', 'CRS_DEP_TIME', 'CRS_ARR_TIME', 'DISTANCE', 'ARR_DELAY_GROUP'],
      dtype='object')

In [332]:
flight_data.ARR_DELAY_GROUP.value_counts()

-1.0    111777
 0.0     38131
 1.0     19410
Name: ARR_DELAY_GROUP, dtype: int64

# Type casting

In [333]:
flight_data.dtypes

FL_DATE              datetime64[ns]
OP_CARRIER                   object
TAIL_NUM                     object
OP_CARRIER_FL_NUM             int64
ORIGIN                       object
DEST                         object
CRS_DEP_TIME                  int64
CRS_ARR_TIME                  int64
DISTANCE                    float64
ARR_DELAY_GROUP             float64
dtype: object

In [334]:
cols_to_cast=["OP_CARRIER_FL_NUM","OP_CARRIER","TAIL_NUM","ORIGIN","DEST","ARR_DELAY_GROUP"]

for i in range(0,len(flight_data.columns)):
    j=flight_data.columns[i]
    if(j in cols_to_cast):
        flight_data[j]=flight_data[j].astype('category')

In [335]:
flight_data["DISTANCE"]=flight_data["DISTANCE"].astype('int')

In [336]:
flight_data["CRS_DEP_TIME"]=flight_data["CRS_DEP_TIME"].astype("datetime64[ns]")
flight_data["CRS_ARR_TIME"]=flight_data["CRS_ARR_TIME"].astype("datetime64[ns]")


### Handling missing values

In [337]:
flight_data.shape

(172194, 10)

In [339]:
flight_data.isnull().sum()

FL_DATE                 0
OP_CARRIER              0
TAIL_NUM              427
OP_CARRIER_FL_NUM       0
ORIGIN                  0
DEST                    0
CRS_DEP_TIME            0
CRS_ARR_TIME            0
DISTANCE                0
ARR_DELAY_GROUP      2876
dtype: int64

In [263]:
flight_data["TAIL_NUM"].isna()

0         False
1         False
2         False
3         False
4         False
          ...  
172189    False
172190    False
172191    False
172192    False
172193    False
Name: TAIL_NUM, Length: 172194, dtype: bool

In [341]:
flight_data["TAIL_NUM"].isnull().sum()

427

In [342]:
flight_data["ARR_DELAY_GROUP"].isnull().sum()

2876

In [343]:
flight_data.dropna(subset=["TAIL_NUM"],axis=0,inplace=True)

In [344]:
flight_data["ARR_DELAY_GROUP"].isnull().sum()

2449

In [345]:
flight_data.dropna(subset=["ARR_DELAY_GROUP"],axis=0,inplace=True)
flight_data.isna().sum()

FL_DATE              0
OP_CARRIER           0
TAIL_NUM             0
OP_CARRIER_FL_NUM    0
ORIGIN               0
DEST                 0
CRS_DEP_TIME         0
CRS_ARR_TIME         0
DISTANCE             0
ARR_DELAY_GROUP      0
dtype: int64

In [346]:
flight_data.dtypes

FL_DATE              datetime64[ns]
OP_CARRIER                 category
TAIL_NUM                   category
OP_CARRIER_FL_NUM          category
ORIGIN                     category
DEST                       category
CRS_DEP_TIME         datetime64[ns]
CRS_ARR_TIME         datetime64[ns]
DISTANCE                      int32
ARR_DELAY_GROUP            category
dtype: object

# Train-Test split

In [347]:
flight_data.OP_CARRIER.nunique()

26

In [348]:
flight_data.ORIGIN.nunique()

372

In [349]:
flight_data.DEST.nunique()

370

In [350]:
flight_data.TAIL_NUM.nunique()

5896

In [352]:
flight_data1 = flight_data.copy()

flight_data1.head()

,FL_DATE,OP_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,CRS_ARR_TIME,DISTANCE,ARR_DELAY_GROUP
0,2019-07-09,ZW,N454AW,3791,AZO,ORD,1970-01-01 00:00:00.000001426,1970-01-01 00:00:00.000001432,122,-1.0
1,2019-07-11,C5,N12195,4895,EWR,ROC,1970-01-01 00:00:00.000001255,1970-01-01 00:00:00.000001414,246,0.0
2,2019-07-23,DL,N841DN,1926,ATL,MCI,1970-01-01 00:00:00.000002148,1970-01-01 00:00:00.000002256,692,0.0
3,2019-07-08,B6,N981JT,1078,LAS,BOS,1970-01-01 00:00:00.000000700,1970-01-01 00:00:00.000001511,2381,-1.0
4,2019-07-28,DL,N807DN,1779,MSP,DEN,1970-01-01 00:00:00.000001151,1970-01-01 00:00:00.000001304,680,-1.0


In [357]:
num_col=pd.DataFrame(flight_data1["DISTANCE"])
num_col

,DISTANCE
0,122
1,246
2,692
3,2381
4,680
...,...
172189,2475
172190,460
172191,2677
172192,139


In [360]:
from sklearn.preprocessing import StandardScaler
s=StandardScaler()

flight_data["DISTANCE"]=s.fit_transform(num_col)
flight_data.head()

,FL_DATE,OP_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,CRS_ARR_TIME,DISTANCE,ARR_DELAY_GROUP
0,2019-07-09,ZW,N454AW,3791,AZO,ORD,1970-01-01 00:00:00.000001426,1970-01-01 00:00:00.000001432,-1.100328,-1.0
1,2019-07-11,C5,N12195,4895,EWR,ROC,1970-01-01 00:00:00.000001255,1970-01-01 00:00:00.000001414,-0.889004,0.0
2,2019-07-23,DL,N841DN,1926,ATL,MCI,1970-01-01 00:00:00.000002148,1970-01-01 00:00:00.000002256,-0.128919,0.0
3,2019-07-08,B6,N981JT,1078,LAS,BOS,1970-01-01 00:00:00.000000700,1970-01-01 00:00:00.000001511,2.749518,-1.0
4,2019-07-28,DL,N807DN,1779,MSP,DEN,1970-01-01 00:00:00.000001151,1970-01-01 00:00:00.000001304,-0.149370,-1.0


In [ ]:
flight_data.groupby(["OP_CARRIER","ARR_DELAY_GROUP"]).plot(kind="bar")